In [ ]:
# Dog Vs Cat dataset couldn't be loaded in the Google Colab 
# as RAM exhausted

# x_train = x_train / 255.0
# model.fit(x_train, y_train) 
# at time particular time, model has to load complete data + weights + baises ... 

# Example - 100M images ... 
# 3MB * 100M < ~$1M

# 60:40 
# 60:40 

In [ ]:
# Keras will take care of all things ... 
# But Keras also, has some expectation from "you" .. 
# train_folder 
#   | - category_1
#       | - images_of_cat-1 
#   | - category_2
#       | - images_of_cat-2
#   | - category_3 
#       | - images_of_cat-3
#   | - category_4 
#       | - images_of_cat-4

# Example DogsVsCat 
# train_data
#   | - cat
#       | - images_of_cat
#   | - dog
#       | - images_of_dog

In [27]:
! nvidia-smi

Sat Feb 19 16:14:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    74W / 149W |    403MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
! mkdir -p /root/.kaggle
! mv kaggle.json /root/.kaggle

In [2]:
! kaggle competitions download -c dogs-vs-cats

 97% 528M/543M [00:03<00:00, 180MB/s]
100% 543M/543M [00:03<00:00, 180MB/s]
 94% 256M/271M [00:01<00:00, 172MB/s]
100% 271M/271M [00:01<00:00, 171MB/s]
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 87.7MB/s]


In [3]:
! unzip -q train.zip

In [ ]:
import os
os.listdir("train")

In [5]:
os.mkdir("train_data")

In [6]:
# Creating category folder 
os.mkdir("train_data/cat")
os.mkdir("train_data/dog")

In [10]:
src_folder = "train/"
dst_cat_folder = "train_data/cat/"
dst_dog_folder = "train_data/dog/"

In [12]:
# shutil.copy("train/cat.0.jpg", "train_data/cat")
import shutil
for imagename in os.listdir(src_folder):
  if "cat" in imagename:
    shutil.copy(src_folder + imagename, dst_cat_folder)
  elif "dog" in imagename:
    shutil.copy(src_folder + imagename, dst_dog_folder)
  else:
    print(imagename)

In [ ]:
os.listdir("train_data/cat")

In [14]:
len(os.listdir("train_data/dog"))

12500

In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [41]:
idg = ImageDataGenerator(rescale=1/255.0, validation_split=0.1)

In [42]:
batch_size = 128

In [43]:
train_gen = idg.flow_from_directory(directory = "train_data", 
                                    target_size=(150, 150), 
                                    batch_size=batch_size, subset="training")

Found 22500 images belonging to 2 classes.


In [44]:
val_gen = idg.flow_from_directory(directory = "train_data", 
                                    target_size=(150, 150), 
                                    batch_size=batch_size, 
                                    subset="validation")

Found 2500 images belonging to 2 classes.


In [ ]:
# x_train -> Read the images, Resize , x_train /255.0

In [46]:
# del model

In [47]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(150, 150, 3), 
                                name="InputLayer"))
model.add(tf.keras.layers.Flatten(name="FlatLayer"))
model.add(tf.keras.layers.Dense(units=100, 
                                activation=tf.keras.activations.relu, 
                                name="Dense1"))
model.add(tf.keras.layers.Dense(units=2, 
                                activation=tf.keras.activations.softmax, 
                                name="OutLayer"))

In [48]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 FlatLayer (Flatten)         (None, 67500)             0         
                                                                 
 Dense1 (Dense)              (None, 100)               6750100   
                                                                 
 OutLayer (Dense)            (None, 2)                 202       
                                                                 
Total params: 6,750,302
Trainable params: 6,750,302
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [49]:
model.compile(optimizer=tf.keras.optimizers.SGD(), 
              loss=tf.keras.losses.categorical_crossentropy, 
              metrics=["acc"])

In [50]:
model.fit(train_gen, validation_data=val_gen, epochs=10)

Epoch 1/10
176/176 [==============================] - 77s 438ms/step - loss: 0.8089 - acc: 0.5296 - val_loss: 0.6831 - val_acc: 0.5572
Epoch 2/10
176/176 [==============================] - 76s 434ms/step - loss: 0.6718 - acc: 0.5858 - val_loss: 0.6661 - val_acc: 0.5920
Epoch 3/10
176/176 [==============================] - 76s 434ms/step - loss: 0.6617 - acc: 0.6049 - val_loss: 0.6610 - val_acc: 0.6120
Epoch 4/10
176/176 [==============================] - 76s 435ms/step - loss: 0.6543 - acc: 0.6139 - val_loss: 0.6530 - val_acc: 0.6060
Epoch 5/10
176/176 [==============================] - 77s 435ms/step - loss: 0.6474 - acc: 0.6168 - val_loss: 0.6605 - val_acc: 0.6076
Epoch 6/10
176/176 [==============================] - 76s 431ms/step - loss: 0.6441 - acc: 0.6247 - val_loss: 0.6539 - val_acc: 0.6172
Epoch 7/10
176/176 [==============================] - 75s 428ms/step - loss: 0.6413 - acc: 0.6299 - val_loss: 0.6489 - val_acc: 0.6124
Epoch 8/10
176/176 [==============================] - 7